<a href="https://colab.research.google.com/github/nisarahamedk/kaggle-riid/blob/master/notebooks/RIID_TF_Transformers_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### RIID Transformer on TPU

In [1]:
import pickle

import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold


np.random.seed(42)
tf.random.set_seed(42)


### *Device* Settings - This needs to be at the TOP¶

In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # no parameter needed for TPU_NAME env variable is set. This is the case for Kaggle
    print("Running on TPU: ", tpu.master())
except ValueError:
    tpu = None

In [3]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default strategy with the available hw
    strategy = tf.distribute.get_strategy()
    
REPLICAS = strategy.num_replicas_in_sync
print("REPLICAS: ", REPLICAS)

REPLICAS:  1


### Datasets

In [4]:
DATA_PATH = 'gs://kds-0d2ffe2dbc91ac5f57c8846e2d8c3ef3f2c48c5e264ae5abb3f92918'

In [5]:
n_train_files = len(tf.io.gfile.glob(DATA_PATH + "/tfrec*"))
n_train_files

32

In [6]:
FOLDS = 10

kfold = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
folds_list = list(kfold.split(np.arange(n_train_files)))
folds_list[:2]

[(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18,
         19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31]),
  array([15, 17, 24, 29])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31]),
  array([ 8,  9, 25, 30]))]

In [7]:
FOLD = 5

train_folds, valid_folds = folds_list[FOLD]
len(train_folds), len(valid_folds)

(29, 3)

In [8]:
train_files = tf.io.gfile.glob([DATA_PATH + "/tfrec_%d.tfrec" % idx for idx in train_folds])
valid_files = tf.io.gfile.glob([DATA_PATH + "/tfrec_%d.tfrec" % idx for idx in valid_folds])

In [9]:
len(train_files), len(valid_files)

(29, 3)

#### Load TFRecord Datasets

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
feature_desc = {
    "content_id": tf.io.FixedLenFeature([], tf.string),
    "task_container_id": tf.io.FixedLenFeature([], tf.string),
    "prior_question_elapsed_time": tf.io.FixedLenFeature([], tf.string),
    "part": tf.io.FixedLenFeature([], tf.string),
    "prev_answered_correctly": tf.io.FixedLenFeature([], tf.string),
    "answered_correctly": tf.io.FixedLenFeature([], tf.string),
}

def parse_example(example):
  example = tf.io.parse_single_example(example, feature_desc)

  content_id = tf.io.parse_tensor(example["content_id"], tf.int16)
  task_container_id = tf.io.parse_tensor(example["task_container_id"], tf.int16)
  prior_question_elapsed_time = tf.io.parse_tensor(example["prior_question_elapsed_time"], tf.float32)
  part = tf.io.parse_tensor(example["part"], tf.int16)
  prev_answered_correctly = tf.io.parse_tensor(example["prev_answered_correctly"], tf.int8)
  answered_correctly = tf.io.parse_tensor(example["answered_correctly"], tf.int8)
  
  return tf.stack([
      tf.cast(content_id, tf.float32),
      tf.cast(task_container_id, tf.float32),
      tf.cast(prior_question_elapsed_time, tf.float32),
      tf.cast(part, tf.float32),
      tf.cast(prev_answered_correctly, tf.float32),
      tf.cast(answered_correctly, tf.float32)
  ]) # [features, seq_len] # TODO make it [seq_len, features] so that we dont have to reshape in transformer

In [12]:
def load_dataset_from_tfrecord(filenames, ds_type="train", cache_to=None):
        # Since we are reading dataset from multiple files. and we dont care about the order.
        # set deterministic reading to False.
        ignore_order = tf.data.Options()
        if ds_type == "train":
            ignore_order.experimental_deterministic = False
            
        dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
        if not cache_to:
            dataset = dataset.cache() # cache to RAM
        else:
            dataset = dataset.cache(cache_to) # cache to file given by self.cache_to 
        if ds_type == "train":
            dataset = dataset.repeat() # repeat individual item, so that we have full batch at every step.
        dataset.with_options(ignore_order)
        dataset = dataset.map(parse_example, num_parallel_calls=AUTOTUNE)
        return dataset

In [13]:
dataset = load_dataset_from_tfrecord(train_files)

In [14]:
SEQ_LEN = 128

In [15]:
@tf.function
def pad(a, seq_len, max_seq_len):
  s = max_seq_len - seq_len
  # making [[0, 0], [s, 0]]
  r = tf.stack([s, tf.constant(0)])
  t = tf.stack([tf.constant([0, 0]), r])
  
  return tf.pad(a, t) # ,1 to debug

@tf.function
def trim(a, seq_len,  max_seq_len):
  start = tf.squeeze(tf.random.uniform((1,), maxval=(seq_len-max_seq_len), dtype=tf.int32))
  # https://www.quora.com/How-does-tf-slice-work-in-TensorFlow
  begin = tf.stack([tf.constant(0), start])
  size = tf.stack([tf.shape(a)[0], max_seq_len])
  
  return tf.slice(a, begin, size) # , start - to debug

@tf.function
def pad_or_trim(a):
  seq_len = tf.shape(a)[-1]
  max_seq_len = SEQ_LEN
  fn = tf.cond(tf.less_equal(seq_len, max_seq_len), lambda: pad(a, seq_len, max_seq_len), lambda: trim(a, seq_len, max_seq_len))
  return fn

In [16]:
dataset = dataset.map(pad_or_trim, num_parallel_calls=AUTOTUNE) # every sample is padded if len < SEQ_LEN or randomly trimmed to SEQ_LEN

In [17]:
@tf.function
def create_padding_mask(seq):
  seq = tf.cast(tf.reduce_all(tf.math.equal(seq, 0), axis=-1), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

@tf.function
def split_x_y_mask(xy):
  x = xy[:-1, :]
  x = tf.transpose(x, (1, 0)) # [seq_len, n_features]
  y = xy[-1, :]
  pad_mask = tf.cast(tf.math.reduce_any(tf.math.not_equal(x, 0), axis=-1), dtype=tf.float32)
  return x, tf.expand_dims(y, axis=-1), tf.expand_dims(pad_mask, axis=-1)

In [18]:
dataset = dataset.map(split_x_y_mask) # x and y

In [19]:
for x, y, pad_mask in dataset.take(1):
  print(x.shape)
  print(y.shape)
  print(pad_mask.shape)

(128, 5)
(128, 1)
(128, 1)


In [20]:
tf.squeeze(pad_mask, axis=-1)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [21]:
tf.squeeze(y, axis=-1)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 0., 0., 1.], dtype=float32)>

In [22]:
dataset = dataset.shuffle(int(1024 * REPLICAS))

In [23]:
BATCH_SIZE = 256

In [24]:
dataset = dataset.batch(BATCH_SIZE)

In [25]:
for xb, yb, mb in dataset.take(1):
  print(xb.shape)
  print(yb.shape)
  print(mb.shape)

(256, 128, 5)
(256, 128, 1)
(256, 128, 1)


In [26]:
# prev_answered_feature
xb[0, :, -1]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 1., 2., 1., 1., 1., 2., 1., 2., 2., 2., 2., 1., 2., 2., 2.,
       2., 2., 1., 2., 2., 2., 2., 2., 1., 2., 2., 1., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 1., 2., 2., 2., 1.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2.,
       2., 1., 2., 2., 2., 2., 2., 2., 1.], dtype=float32)>

In [27]:
# corresponding mask
mb[0, :, 0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [28]:
dataset = dataset.prefetch(AUTOTUNE)

In [29]:
# len(list(iter(dataset.take(1000)))) # dataset repeats indefinitely

### Model

##### Positional Encoding

In [30]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [31]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

##### Look ahead mask

In [32]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

##### Scaled Dot Product Attention

In [33]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

##### Multi Head Attention

In [34]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

##### Pointwise FeedForward Network

In [35]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

##### EncoderLayer

In [36]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

##### Encoder

In [37]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.content_id_emb = tf.keras.layers.Embedding(embed_size_dict["content_id"] + 1, d_model)
    self.task_container_id_emb = tf.keras.layers.Embedding(embed_size_dict["task_container_id"] + 1, d_model)
    self.part_emb = tf.keras.layers.Embedding(embed_size_dict["part"] + 2, d_model)
    self.prior_question_elapsed_time_emb = tf.keras.layers.Dense(d_model, use_bias=True)
    self.prev_answered_emb = tf.keras.layers.Embedding(4, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)


    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embeddings and position encoding.
    c_emb = self.content_id_emb(x[..., 0])  # (batch_size, input_seq_len, d_model)
    t_emb = self.task_container_id_emb(x[..., 1])
    prior_time_emb = self.prior_question_elapsed_time_emb(tf.expand_dims(x[..., 2], axis=-1))
    pt_emb = self.part_emb(x[..., 3])
    pv_emb = self.prev_answered_emb(x[..., 4])
    x = c_emb + t_emb + prior_time_emb + pt_emb + pv_emb
    
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask) # (batch_size, input_seq_len, d_model)

    return x

In [38]:
class TransformerSeq2SeqClassifier(keras.models.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate=0.1):
    super(TransformerSeq2SeqClassifier, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate)
    self.out = tf.keras.layers.Dense(1, activation="sigmoid")

  def call(self, x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = create_look_ahead_mask(seq_len)
    encoded = self.encoder(x, mask=look_ahead_mask)

    out = self.out(encoded)
    return out # [batch_size, input_seq_len, 1]


###### Embedding Sizes

In [39]:
embed_sizes = pickle.loads(tf.io.read_file(DATA_PATH + "/emb_sz.pkl").numpy())

In [40]:
embed_sizes

{'content_id': 32736, 'part': 7, 'task_container_id': 9999}

In [41]:
with strategy.scope():
  model = TransformerSeq2SeqClassifier(
      num_layers=1,
      d_model=512,
      num_heads=8,
      dff=1024,
      maximum_position_encoding=128,
      embed_size_dict=embed_sizes
  )

In [42]:
for xb, yb, mb in dataset.take(1):
  print(xb.shape, yb.shape, mb.shape)

(256, 128, 5) (256, 128, 1) (256, 128, 1)


In [43]:
# y_pred = model(xb)

In [44]:
# y_pred.shape

#### Training

In [45]:
with strategy.scope():
  optimizer = keras.optimizers.Adam(learning_rate=3e-3)
  loss = keras.losses.BinaryCrossentropy()

In [46]:
with strategy.scope():
  model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy", keras.metrics.AUC()], weighted_metrics=["accuracy", keras.metrics.AUC()]) # "weighted_metrics" not supported on TPU

In [ ]:
### OVERFIT SINGLE BATCH
with strategy.scope():
  x, y, mask = next(iter(dataset.take(1))) # cannot just use take(), cause that return different batch everytime
  print(x.shape, y.shape, mask.shape)
  model.fit(x, y, epochs=100, sample_weight=mask)

(256, 128, 5) (256, 128, 1) (256, 128, 1)
Epoch 1/100
8/8 [==============================] - 2s 215ms/step - loss: 1.7062 - accuracy: 0.3794 - auc: 0.4812 - weighted_accuracy: 0.5543 - auc_1: 0.4879
Epoch 2/100
8/8 [==============================] - 2s 202ms/step - loss: 0.3871 - accuracy: 0.4573 - auc: 0.5310 - weighted_accuracy: 0.5417 - auc_1: 0.5035
Epoch 3/100
8/8 [==============================] - 2s 204ms/step - loss: 0.3720 - accuracy: 0.4109 - auc: 0.4908 - weighted_accuracy: 0.5514 - auc_1: 0.4914
Epoch 4/100
8/8 [==============================] - 2s 202ms/step - loss: 0.3664 - accuracy: 0.3837 - auc: 0.5236 - weighted_accuracy: 0.5900 - auc_1: 0.5083
Epoch 5/100
8/8 [==============================] - 2s 197ms/step - loss: 0.3599 - accuracy: 0.3319 - auc: 0.5151 - weighted_accuracy: 0.6162 - auc_1: 0.4999
Epoch 6/100
8/8 [==============================] - 2s 201ms/step - loss: 0.3582 - accuracy: 0.3316 - auc: 0.5373 - weighted_accuracy: 0.6164 - auc_1: 0.5081
Epoch 7/100
8/8 

In [144]:
with strategy.scope():
  model.fit(dataset, epochs=100, steps_per_epoch=10)

Epoch 1/100
10/10 [==============================] - 7s 687ms/step - loss: 0.2948 - accuracy: 0.8457 - auc: 0.9253 - weighted_accuracy: 0.6967 - auc_1: 0.7401
Epoch 2/100
10/10 [==============================] - 7s 694ms/step - loss: 0.3044 - accuracy: 0.8419 - auc: 0.9218 - weighted_accuracy: 0.6994 - auc_1: 0.7396
Epoch 3/100
10/10 [==============================] - 7s 690ms/step - loss: 0.2972 - accuracy: 0.8447 - auc: 0.9251 - weighted_accuracy: 0.6987 - auc_1: 0.7466
Epoch 4/100
10/10 [==============================] - 7s 693ms/step - loss: 0.3008 - accuracy: 0.8429 - auc: 0.9241 - weighted_accuracy: 0.7000 - auc_1: 0.7476
Epoch 5/100
10/10 [==============================] - 7s 692ms/step - loss: 0.2923 - accuracy: 0.8486 - auc: 0.9281 - weighted_accuracy: 0.7055 - auc_1: 0.7546
Epoch 6/100
10/10 [==============================] - 7s 701ms/step - loss: 0.2976 - accuracy: 0.8448 - auc: 0.9260 - weighted_accuracy: 0.7033 - auc_1: 0.7508
Epoch 7/100
10/10 [===========================

KeyboardInterrupt: ignored